Spark project

In [1]:
# !pip install praw
# !pip install textblob
# !pip install pyspark

In [2]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from textblob import TextBlob

In [3]:
# Create a SparkContext
sc = SparkContext(appName="RedditStreamingSentimentAnalysis")

your 131072x1 screen size is bogus. expect trouble
23/04/20 15:08:03 WARN Utils: Your hostname, DESKTOP-CBR75GN resolves to a loopback address: 127.0.1.1; using 172.21.23.51 instead (on interface eth0)
23/04/20 15:08:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/20 15:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
sc.setLogLevel("ERROR")

In [5]:
# Create a StreamingContext with a batch interval of 10 seconds
ssc = StreamingContext(sc, 5)

/home/maxime/Scientific_Programming/testvenv/lib/python3.8/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(


In [6]:
# Create a DStream from the socket
dstream = ssc.socketTextStream("localhost", 8888)

In [7]:

# Define a function to perform sentiment analysis on each message
def analyze_sentiment(message):
    blob = TextBlob(message)
    polarity = blob.sentiment.polarity
    subjectivity = blob.sentiment.subjectivity
    if polarity > 0:
        sentiment = "positive"
    elif polarity < 0:
        sentiment = "negative"
    else:
        sentiment = "neutral"
    return (message,sentiment)
    # return (sentiment,1)

In [8]:
# Apply the sentiment analysis function to each message in the DStream
tagged_dstream = dstream.filter(lambda line: len(line) > 0)\
                        .map(lambda message: analyze_sentiment(message))\
                        # .reduceByKey(lambda a, b: a + b)
# .flatMap(lambda line: line.split(" "))\
# .map(lambda line: line.split("|"))\

# Print the results
tagged_dstream.pprint()


In [9]:
# # For debugging purposes (to check if we receive correctly the data)
# counts = dstream.flatMap(lambda line: line.split(" "))\
#                 .filter(lambda word: len(word) > 0)\
#                 .map(lambda word: (word, 1))\
#                 .reduceByKey(lambda a, b: a + b)
# counts.pprint()

In [10]:
# Start the streaming context
ssc.start()
# Wait for the streaming to finish
ssc.awaitTermination()

-------------------------------------------
Time: 2023-04-20 15:08:10
-------------------------------------------
("I'm wondering if anyone has any recommendations for virtual python coding camps for kids? My son has been working through some books and is having success but I wouldn't mind putting him in front of an instructor of some sort as well.", 'positive')



-------------------------------------------
Time: 2023-04-20 15:08:15
-------------------------------------------
('Thanks in advance! I mean no offence as I’ve certainly never created a package before - but can you explain how this differs from applying some simple regex? y spoiler tho? Definitely going to have a look at this! Many thanks! There are already a lot of utilities (many of which are built into most operating systems) that do this. Why re-invent the wheel as a third party python package? Thanks for your comment, actually now I’m using regex, the difference now is I’m using also request lib to handle connection and convert it to text, also I’m using KMP algorithm to search for patterns and words.', 'positive')
("This is the beta version, I will keep adding more algorithms and more features like save in file, analytics and more, but I want to take step one and publish it In case you haven't gotten that far in Python yet In case you mean it is simple, yes it is simple package 

-------------------------------------------
Time: 2023-04-20 15:08:20
-------------------------------------------
("My god... I dunno, I only skimmed the sales pitch when our infosec team asked us to integrate with it. I don't know the pricing model or anything ;) > 12. Strive to make a function not return any arguments", 'positive')
('I do this in OO programming. But this is a big no no in functional programming What a ton of bullpoo from the 90-es!', 'neutral')
('> 6. Strive to avoid or minimize the number of function arguments', 'neutral')
('> 12. Strive to make a function not return any arguments', 'neutral')
("== Make sure you can't have even a vague idea what function does until you thoroughly read it through. ", 'neutral')
('> 7. Error handling blocks (try/catch) should be placed in separate functions', 'neutral')
('== Force stuff into another function just to avoid 1 level of indentation?', 'neutral')
('> 9. If possible, replace tests with exception handling', 'neutral')
('... 

-------------------------------------------
Time: 2023-04-20 15:08:25
-------------------------------------------
('Yup :). Thanks! I’ve never heard of loguru before. When I’m not using just the standard lib stuff, I reach for [structlog](https://www.structlog.org/en/stable/). Approachable, can sit right on top of standard logger and has other nifty features too. Never heard of it, is this like "pretend its super popular and everyone believes its popular" ? ', 'positive')
("But then again I am still at from logging import logger Structured logging make it much easier to parse and search logs. Go even introduce an official structured logging package in 1.21: https://www.reddit.com/r/golang/comments/11sdqia/slog\\_proposal\\_accepted\\_for\\_go\\_121/ reads more like a guerrilla advertising for this library. never heard of it. We use import logging and structlog. I’ve seen it used mostly in production rather than in open source projects. Perhaps this is the answer? People probably find o

-------------------------------------------
Time: 2023-04-20 15:08:30
-------------------------------------------
('They are not mutually exclusive. Yes, as much as this tutorial adds to anything. Automatic CLI generation from class attributes, ships with a logging attribute, can be configured via CLI or files or OO programming, reactive objects with observers. And it\'s what ipython is written in. I\'ve written a few posts at my blog about them, here is one https://may69.com/pros-dont-write-scripts/ tkinter canvas, pyside6 canvas, or, if want it really easy, Turtle. Not sure what kind of geometrical pictures you\'re thinking of, but if it\'s solid modeling, look at SolidPython on PyPI. Pillow I use [drawsvg](https://pypi.org/project/drawsvg/) Check out TikZ I swr to god these days anything will get some chatgpt bs thrown at it. "Combines elements if excel, jupyter notebooks and chatgpt" what does that even mean? Because of my frustration with pandas, I created op. Is this something an

-------------------------------------------
Time: 2023-04-20 15:08:35
-------------------------------------------
('The practical effect is that Streamlit is really great for simple dashboards because it requires very little effort to get things working. What specific issues have you encountered? I quick search I found Unity, Unreal and Blender have graphic generators ', 'positive')
("I see you've posted a GitHub link to a Jupyter Notebook! GitHub doesn't ", 'neutral')
('render large Jupyter Notebooks, so just in case, here is an ', 'positive')
('[nbviewer](https://nbviewer.jupyter.org/) link to the notebook:', 'neutral')
('https://nbviewer.jupyter.org/url/github.com/ArtificialIntelligenceToolkit/aitk/blob/master/notebooks/Derivation%20of%20the%20Y-Combinator.ipynb', 'neutral')
('Want to run the code yourself? Here is a [binder](https://mybinder.org/) ', 'neutral')
('link to start your own Jupyter server and try it out!', 'positive')
('https://mybinder.org/v2/gh/ArtificialIntelligenceT

-------------------------------------------
Time: 2023-04-20 15:08:40
-------------------------------------------



ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/maxime/Scientific_Programming/testvenv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/maxime/Scientific_Programming/testvenv/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


-------------------------------------------
Time: 2023-04-20 15:08:45
-------------------------------------------



KeyboardInterrupt: 